<h1>Logistic Regression</h1>

Notebook Goals

* Learn how to create a logistic regression model using scikit-learn

<h2> What are some advantages of logistic regression?</h2> 

How do you create a logistic regression model using Scikit-Learn? The first thing you need to know is that despite the name logistic regression containing the word regression, logistic regression is a model used for classification. Classification models can be used for tasks like classifying flower species or image recognition. All of this of course depends on the availability and quality of your data. Logistic Regression has some advantages including

* Model training and predictions are relatively fast
* No tuning is usually needed for logistic regression unless you want to regularize your model. 
* Finally, it can perform well with a small number of observations. 

<h2> Import Libraries</h2>

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from IPython.display import Video
from matplotlib.ticker import FormatStrFormatter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

from sklearn import metrics

## Load the Dataset
The Iris dataset is one of datasets scikit-learn comes with that do not require the downloading of any file from some external website. The code below loads the iris dataset.

In [ ]:
df = pd.read_csv('data/modifiedIris2Classes.csv')

In [ ]:
df.head()

<h2>  Remove Missing or Impute Values </h2>
If you want to build models with your data, null values are (almost) never allowed. It is important to always see how many samples have missing values and for which columns.

In [ ]:
# Look at the shape of the dataframe
df.shape

In [ ]:
# There is a missing value in the Length column which is a feature
df.isnull().sum()

<h2> Train Test Split </h2>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['petal length (cm)']], df['target'], random_state=0)

<h2> Standardize the Data</h2>
Logistic Regression is effected by scale so you need to scale the features in the data before using Logistic Regresison. You can transform the data onto unit scale (mean = 0 and variance = 1) for better performance. Scikit-Learn's `StandardScaler` helps standardize the dataset’s features. Note you fit on the training set and transform on the training and test set.

In [ ]:
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(X_train)

# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

<h2>Logistic Regression</h2>

<b>Step 1:</b> Import the model you want to use

In sklearn, all machine learning models are implemented as Python classes

In [ ]:
# This was already imported earlier in the notebook so commenting out
#from sklearn.linear_model import LogisticRegression

<b>Step 2:</b> Make an instance of the Model

This is a place where we can tune the hyperparameters of a model. Typically this is where you tune C which is related to regularization

In [ ]:
clf = LogisticRegression()

<b>Step 3:</b> Training the model on the data, storing the information learned from the data

Model is learning the relationship between x (features sepal width, sepal height etc) and y (labels-which species of iris)

In [ ]:
clf.fit(X_train, y_train)

<b>Step 4:</b> Predict the labels of new data (new flowers)

Logistic regression also allows you to see prediction probabilities as well as  a prediction. This is not like other algorithms like decision trees for classification which only give you a prediction not a probability. 

In [ ]:
# One observation's petal length after standardization
X_test[0].reshape(1,-1)

In [ ]:
print('prediction', clf.predict(X_test[0].reshape(1,-1))[0])
print('probability', clf.predict_proba(X_test[0].reshape(1,-1)))

If this is unclear, let's visualize how logistic regression makes predictions by looking at our test data!

In [ ]:
example_df = pd.DataFrame()
example_df.loc[:, 'petal length (cm)'] = X_test.reshape(-1)
example_df.loc[:, 'target'] = y_test.values
example_df['logistic_preds'] = pd.DataFrame(clf.predict_proba(X_test))[1]

In [ ]:
example_df.head()

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,7));


virginicaFilter = example_df['target'] == 1
versicolorFilter = example_df['target'] == 0

ax.scatter(example_df.loc[virginicaFilter, 'petal length (cm)'].values,
            example_df.loc[virginicaFilter, 'logistic_preds'].values,
           color = 'g',
           s = 60,
           label = 'virginica')


ax.scatter(example_df.loc[versicolorFilter, 'petal length (cm)'].values,
            example_df.loc[versicolorFilter, 'logistic_preds'].values,
           color = 'b',
           s = 60,
           label = 'versicolor')

ax.axhline(y = .5, c = 'y')

ax.axhspan(.5, 1, alpha=0.05, color='green')
ax.axhspan(0, .4999, alpha=0.05, color='blue')
ax.text(0.5, .6, 'Classified as viginica', fontsize = 16)
ax.text(0.5, .4, 'Classified as versicolor', fontsize = 16)

ax.set_ylim(0,1)
ax.legend(loc = 'lower right', markerscale = 1.0, fontsize = 12)
ax.tick_params(labelsize = 18)
ax.set_xlabel('petal length (cm)', fontsize = 24)
ax.set_ylabel('probability of virginica', fontsize = 24)
ax.set_title('Logistic Regression Predictions', fontsize = 24)
fig.tight_layout()


<h2> Measuring Model Performance</h2> 

While there are other ways of measuring model performance (precision, recall, F1 Score, ROC Curve, etc), let's keep this simple and use accuracy as our metric. 
To do this are going to see how the model performs on new data (test set)

Accuracy is defined as:
(fraction of correct predictions): correct predictions / total number of data points

In [ ]:
score = clf.score(X_test, y_test)
print(score)

Accuracy is one metric, but it doesn't say give much insight into what was wrong. Let's look at a confusion matrix

In [ ]:
cm = metrics.confusion_matrix(y_test, clf.predict(X_test))

plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True,
            fmt=".0f",
            linewidths=.5,
            square = True,
            cmap = 'Blues');
plt.ylabel('Actual label', fontsize = 17);
plt.xlabel('Predicted label', fontsize = 17);
plt.title('Accuracy Score: {}'.format(score), size = 17);
plt.tick_params(labelsize= 15)

<h2>What went wrong with the confusion matrix? It looks bad!</h2>

In [ ]:
cm = metrics.confusion_matrix(y_test, clf.predict(X_test))

plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True,
            fmt=".0f",
            linewidths=.5,
            square = True,
            cmap = 'Blues');
plt.ylabel('Actual label', fontsize = 17);
plt.xlabel('Predicted label', fontsize = 17);
plt.title('Accuracy Score: {}'.format(score), size = 17);
plt.tick_params(labelsize= 15)

# You can comment out the next 4 lines if you like
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values

Let's look at the same information in a table in a clearer way. 

In [ ]:
# ignore this code

modified_cm = []
for index,value in enumerate(cm):
    if index == 0:
        modified_cm.append(['TN = ' + str(value[0]), 'FP = ' + str(value[1])])
    if index == 1:
        modified_cm.append(['FN = ' + str(value[0]), 'TP = ' + str(value[1])])   
   

In [ ]:
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=np.array(modified_cm),
            fmt="",
            annot_kws={"size": 20},
            linewidths=.5,
            square = True,
            cmap = 'Blues',
            xticklabels = ['versicolor', 'viginica'],
            yticklabels = ['versicolor', 'viginica'],
            );

plt.ylabel('Actual label', fontsize = 17);
plt.xlabel('Predicted label', fontsize = 17);
plt.title('Accuracy Score: {:.3f}'.format(score), size = 17);
plt.tick_params(labelsize= 15)

# You can comment out the next 4 lines if you like
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values

Notice that the score stops improving after a certain number of estimators (decision trees). One way to get a better score would be to include more features in the features matrix.

## Common questions

<h3>What would happen if you change the prediction threshold from .5 for picking a positive class</h3>

By default, and with respect to the underlying assumptions of logistic regression, we predict a positive class when the probability of the class is greater than .5 and predict a negative class otherwise.

If you changed the prediction threshold from .5 to .2, you would predict more true positives but fewer true negatives. You can see this clearly using <a href="http://mfviz.com/binary-predictions/">this visual by Michael Freeman.</a>

<h3>What is the effect of changing the hyperparameter C?</h3>

Looking at the effect of increasing C if you have `l1` regularization. Smaller values specify stronger regularization. The code below shows this for the Wisconsin breast cancer dataset in an effort to mimic Michael Freeman's visualization

See the following file to look at the effect of changing C

In [ ]:
#Video('imagesanimation/effectOfCLogisticRegression.mp4')

In [ ]:
df = pd.read_csv('data/wisconsinBreastCancer.csv')

In [ ]:
# Same code was earlier in notebook, but here for clarity
# The rest of the lines in this sectionis just code I used to make the animation above

col_names = ['worst_concave_points']

X = df[col_names].values.reshape(-1,1)
y = df['diagnosis']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state = 0)

# Standardize Data
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(X_train)

# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
for index,c in enumerate(np.linspace(-3, 3, num = 25)):
    
    c_value = 10**c
    c_value_str = "{0:0.3f}".format(c_value)
    # Keep in mind that there is l2 penalty by default like we have for ridge regression
    logreg = LogisticRegression(C = c_value)
    
    logreg.fit(X_train, y_train)

    example_df = pd.DataFrame()
    example_df.loc[:, 'worst_concave_points'] = X_train.reshape(-1)
    example_df.loc[:, 'diagnosis'] = y_train.values

    example_df['logistic_preds'] = pd.DataFrame(logreg.predict_proba(X_train))[1]
    example_df = example_df.sort_values(['logistic_preds'])

    plt.scatter(example_df['worst_concave_points'], example_df['diagnosis'])
    plt.plot(example_df['worst_concave_points'], example_df['logistic_preds'].values, color='red')

    plt.ylabel('malignant (1) or benign (0)', fontsize = 13)
    plt.xlabel('worst_concave_points', fontsize = 13)
    plt.title("Logistic Regression (L1) C = " + str(c_value_str), fontsize = 15)
    plt.savefig('imagesanimation/' + 'initial' + str(index).zfill(4) + '.png', dpi = 100)
    plt.cla()

<h3>What is the effect of regularization on accuracy?</h3>

You can look at the video imagesanimation/logisticRegularizationEffectAccuracy.mp4

In [ ]:
# Same code was earlier in notebook, but here for clarity

col_names = ['worst_concave_points']

X = df[col_names].values.reshape(-1,1)
y = df['diagnosis']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state = 0)

# Standardize Data
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(X_train)

# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model_list, coef_list, c_value_list, accuracy_list, example_df_list = [], [], [], [], []

for index,c in enumerate(np.linspace(-3, 3, num = 25)):
    
    c_value = 10**c
    c_value_str = "{0:0.3f}".format(c_value)
    # Keep in mind that there is l2 penalty by default like we have for ridge regression
    logreg = LogisticRegression(C = c_value,
                                penalty = 'l1',
                                solver = 'saga',
                                max_iter = 100000)
                                
    
    logreg.fit(X_train, y_train)
    
    # Subplot (top)
    example_df = pd.DataFrame()
    example_df.loc[:, 'worst_concave_points'] = X_train.reshape(-1)
    example_df.loc[:, 'diagnosis'] = y_train.values

    example_df['logistic_preds'] = pd.DataFrame(logreg.predict_proba(X_train))[1]
    example_df = example_df.sort_values(['logistic_preds'])
    example_df_list.append(example_df)
    model_list.append(logreg)
    accuracy_list.append(logreg.score(X_test, y_test))
    coef_list.append(logreg.coef_[0])
    c_value_list.append(c_value)
        
temp_df = pd.DataFrame(coef_list, index = c_value_list, columns = col_names)
    
temp_df.loc[:, 'model'] = model_list

# Giving the index a name (it is not a column)
temp_df.index.name = 'C (Inverse of Regularization Strength)'

In [ ]:
for index, (c_value,example_df) in enumerate(zip(c_value_list, example_df_list)):

    c_value_str = "{0:0.3f}".format(c_value)

    fig, axes = plt.subplots(nrows = 2,
                             ncols = 1,
                             figsize = (12, 7));

    # Just formatting, not relevant for this class
    fig.subplots_adjust(wspace=0.1, hspace = .55)

    """
    fig.suptitle("Logistic Regression (L1) C = " + str(c_value_str),
                 fontsize = 15,
                 y=.94)
    """

    # Code is just to make it so you have different colors in the "title"
    # https://stackoverflow.com/questions/9350171/figure-title-with-several-colors-in-matplotlib
    fig.text(0.45,
             0.92,
             "Logistic Regression (L1) C = ",
             ha="center",
             va="bottom",
             size=20,
             color="black")

    fig.text(0.68,
             0.92,
             str(c_value_str),
             ha="center",
             va="bottom",
             size=20,
             color="purple",)

    axes[0].scatter(example_df['worst_concave_points'], example_df['diagnosis'])
    axes[0].plot(example_df['worst_concave_points'], example_df['logistic_preds'].values, color='red')
    axes[0].set_ylabel('malignant (1) or benign (0)', fontsize = 13)
    axes[0].set_xlabel('worst_concave_points', fontsize = 11)

    axes[1].plot(temp_df.index,
                 temp_df.loc[:, 'worst_concave_points'],
                 label = 'worst_concave_points',
                 color = 'purple');
    
    axes[1].axvspan(c_value - c_value/10,c_value +  c_value/10, color='orange', alpha=0.3, zorder = 1);

    coefLimits = temp_df.min().min(), temp_df.max().max()
    accuracyLimits = min(accuracy_list), max(accuracy_list)
    
    axes[1].tick_params('y', colors='purple');
    axes[1].set_ylim(coefLimits)
    axes[1].set_yticks(np.linspace(coefLimits[0],coefLimits[1], 11))
    axes[1].set_xscale('log')
    axes[1].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    axes[1].set_ylabel('weights', color='purple', fontsize = 13)
    axes[1].set_xlabel('C', fontsize = 11)

    axesTwin=axes[1].twinx()
    axesTwin.plot(temp_df.index, accuracy_list, color = 'g')
    axesTwin.tick_params('y', colors='g');
    axesTwin.set_ylim(accuracyLimits)
    axesTwin.set_yticks(np.linspace(accuracyLimits[0],accuracyLimits[1], 11))
    axesTwin.set_ylabel('Accuracy', color='g', fontsize = 13); 

    
    axes[1].grid();
    ###
    
    fig.savefig('imagesanimation2/' + 'initial' + str(index).zfill(4) + '.png', dpi = 100)

In [ ]:
# If you are really curious, I can share how this works. 
#!ffmpeg -framerate 1 -i 'initial%04d.png' -c:v libx264 -r 30 -pix_fmt yuv420p initial_002.mp4